In [27]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5

import sys
sys.path.append('..\..')
import matplotlib.pyplot as plt
from utils.behaviour_utils import RBias, CalculateRBiasWindow
import matplotlib
from utils.post_processing_utils import *
from utils.plotting import calculate_error_bars, multi_conditions_plot
from set_global_params import value_change_mice, figure_directory, reproduce_figures_path, spreadsheet_path
from utils.stats import cohen_d_paired
from save_to_excel import save_figure_data_to_excel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Making sure files are where they need to be - don't need to run if you have repro data

In [4]:
# In case the repro data file doesn't exist, copy it over from the raw path 
# (don't run if you don't have the full data set and only have repro data - will throw error)
repro_dir = os.path.join(reproduce_figures_path, 'ED_fig6', 'value_change_behaviour')
mice = value_change_mice['Nacc'] + value_change_mice['tail']
for mouse_num, mouse_id in enumerate(mice):
    all_experiments = get_all_experimental_records()
    sessions = all_experiments[(all_experiments['mouse_id'] == mouse_id) & (all_experiments['experiment_notes'] == 'value switch')]['date'].values
    for date in sessions:
        experiment_to_process = all_experiments[(all_experiments['date'] == date) & (all_experiments['mouse_id'] == mouse_id)]
        copy_behaviour_to_folder_mouse_name(experiment_to_process, source_dir=processed_data_path, target_dir=repro_dir)

# Main Code

In [5]:
def moving_average(x, window=50):
    rolling_average = np.empty(len(x))
    rolling_average[:] = np.nan
    for i in range(window, len(x)):
        win = range((i - window), i)
        rolling_average[i] = np.mean(x[win])
    return rolling_average

In [6]:
repro_dir = os.path.join(reproduce_figures_path, 'ED_fig6', 'value_change_behaviour')

In [7]:
pre_pc = []
post_pc = []
mice = value_change_mice['Nacc']
mouse_ids = []
session_ids = []
moving_avs = []
reward_blocks = []
performance_to_delvalued_tone = []
for mouse_num, mouse_id in enumerate(mice):
    all_experiments = get_all_experimental_records()
    sessions = all_experiments[(all_experiments['mouse_id'] == mouse_id) & (all_experiments['experiment_notes'] == 'value switch')]['date'].values
    for date in sessions:
        experiment_to_process = all_experiments[(all_experiments['date'] == date) & (all_experiments['mouse_id'] == mouse_id)]
        trial_data = open_experiment_just_behaviour(experiment_to_process, root_dir=repro_dir)
        trial_data.loc[trial_data['Trial outcome'] == 3, 'Trial outcome'] = 0
        red_trial_data = trial_data[trial_data['State name'] == 'TrialStart']
        post_trials = red_trial_data[red_trial_data['Trial num'] >= 100]
        pre_trials = red_trial_data[red_trial_data['Trial num'] < 100]
        post_reward_side = post_trials['Reward block'].unique()[0]
        if post_reward_side == 1:
            side = -1
            devalued_trial_type = 7
        elif post_reward_side == 5:
            side = 1
            devalued_trial_type = 1
        devalued_trials = post_trials[post_trials['Trial type'] == devalued_trial_type]
        performance = np.mean(devalued_trials['Trial outcome'].values) * 100
        pre_bias = RBias(pre_trials['First response'], pre_trials['First choice correct'])* side
        post_bias = RBias(post_trials['First response'], post_trials['First choice correct']) *side
        post_pc.append(post_bias)
        pre_pc.append(pre_bias)
        session_ids.append(date)
        mouse_ids.append(mouse_id)
        reward_blocks.append(post_reward_side)
        performance_to_delvalued_tone.append(performance)
        


In [8]:
mice = value_change_mice['tail']
for mouse_num, mouse_id in enumerate(mice):
    all_experiments = get_all_experimental_records()
    sessions = all_experiments[(all_experiments['mouse_id'] == mouse_id) & (all_experiments['experiment_notes'] == 'value switch')]['date'].values
    for date in sessions:
        experiment_to_process = all_experiments[(all_experiments['date'] == date) & (all_experiments['mouse_id'] == mouse_id)]
        trial_data = open_experiment_just_behaviour(experiment_to_process, root_dir=repro_dir)
        trial_data.loc[trial_data['Trial outcome'] == 3, 'Trial outcome'] = 0
        red_trial_data = trial_data[trial_data['State name'] == 'TrialStart']
        post_trials = red_trial_data[red_trial_data['Trial num'] >= 100]
        pre_trials = red_trial_data[red_trial_data['Trial num'] < 100]
        post_reward_side = post_trials['Reward block'].unique()[0]
        if post_reward_side == 1:
            side = -1
            devalued_trial_type = 7
        elif post_reward_side == 5:
            side = 1
            devalued_trial_type = 1
        devalued_trials = post_trials[post_trials['Trial type'] == devalued_trial_type]
        performance = np.mean(devalued_trials['Trial outcome'].values) * 100
        pre_bias = RBias(pre_trials['First response'], pre_trials['First choice correct'])* side
        post_bias = RBias(post_trials['First response'], post_trials['First choice correct']) *side
        post_pc.append(post_bias)
        pre_pc.append(pre_bias)
        session_ids.append(date)
        mouse_ids.append(mouse_id)
        reward_blocks.append(post_reward_side)
        performance_to_delvalued_tone.append(performance)


In [9]:
behavioural_change_performance = {}
behavioural_change_performance['mouse'] = mouse_ids
behavioural_change_performance['session'] = session_ids
behavioural_change_performance['performance to devalued tone'] = performance_to_delvalued_tone 
behavioural_change_performance_df = pd.DataFrame(behavioural_change_performance)

per_mouse_performance = behavioural_change_performance_df.groupby(['mouse'])[['performance to devalued tone']].apply(np.mean)
per_mouse_performance = per_mouse_performance.reset_index()

C:\Users\francescag\AppData\Local\Continuum\miniconda3\envs\py38\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [10]:
np.mean(per_mouse_performance['performance to devalued tone'])

81.9938010150495

In [11]:
np.std(per_mouse_performance['performance to devalued tone'])

6.239987618045049

In [12]:
behavioural_change = {}
behavioural_change['mouse'] = mouse_ids
behavioural_change['session'] = session_ids
behavioural_change['pre bias'] = pre_pc 
behavioural_change['post bias'] = post_pc
behavioural_change_df = pd.DataFrame(behavioural_change)

In [13]:
df_for_plot = behavioural_change_df.groupby(['mouse'])[['pre bias', 'post bias']].apply(np.mean)*100
df_for_plot = df_for_plot.reset_index()

C:\Users\francescag\AppData\Local\Continuum\miniconda3\envs\py38\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [14]:
df_for_plot1 = df_for_plot.set_index('mouse').transpose()


In [18]:
comparison_csv = os.path.join(spreadsheet_path, 'ED_fig6', 'ED_fig6U_bias_to_big_value_change.csv')
if not os.path.exists(comparison_csv):
    (df_for_plot1.T).to_csv(comparison_csv)


In [15]:
font = {'size': 7}
matplotlib.rc('font', **font)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['font.sans-serif'] = 'Arial'
matplotlib.rcParams['font.family']


fig, ax = plt.subplots(figsize=[2,2])
multi_conditions_plot(ax, df_for_plot1, mean_line_color='#7FB5B5', mean_linewidth=6, show_err_bar=False)
plt.xticks([0, 1], ['equal\nrewards', 'unequal\nrewards'])
plt.ylabel('Bias towards BIG side (%)')

y = df_for_plot1.to_numpy().max() + .2
h = 1
plt.plot([0, 0, 1, 1], [y, y+h, y+h, y],c='k',lw=1)
ax.text(.5, y+h, '**', ha='center', fontsize=8)
plt.tight_layout()
figure_dir = figure_directory
plt.tight_layout()
plt.show()

In [12]:
import scipy.stats as stats
pre_data = df_for_plot1.T['pre bias'].values
post_data = df_for_plot1.T['post bias'].values
stat, pval = stats.ttest_rel(pre_data, post_data)
pval

0.0017325467666258832

In [14]:
cohen_d_paired(post_data, pre_data)

cohen d:  1.3898675116302859


1.3898675116302859

In [18]:
np.mean(df_for_plot1.T['post bias'].values)

5.688087278321691

In [17]:
np.mean(100 - (df_for_plot1.T['post bias'].values + 50))

44.31191272167831

In [110]:
differences = pre_data - post_data
stats.shapiro(differences)


ShapiroResult(statistic=0.9447067379951477, pvalue=0.6064687371253967)

In [56]:
df_for_plot1.to_numpy().max() + .2

11.697722384256606

In [19]:
def calc_per_mouse_mean(mouse_moving_avs):
    num_sessions = len(mouse_moving_avs)
    num_trials = [len(m) for m in mouse_moving_avs]
    min_num_trials = min(num_trials)
    all_sessions = np.empty((num_sessions, min_num_trials))
    all_sessions[:] = np.nan
    for i, session_data in enumerate(mouse_moving_avs):
        all_sessions[i, :min_num_trials] = session_data[:min_num_trials]
    moving_av = np.nanmean(all_sessions, axis=0)
    return moving_av

In [20]:
def align_mulitple_mice_moving_avs(mice, moving_avs):
    num_mice = len(mice)
    num_trials = [len(m) for m in moving_avs]
    min_num_trials = min([len(m) for m in moving_avs])
    all_mice = np.empty((num_mice, min_num_trials))
    all_mice[:] = np.nan
    for i, mouse_data in enumerate(moving_avs):
        all_mice[i, :min_num_trials] = mouse_data[:min_num_trials]
    return all_mice, min_num_trials

In [21]:

moving_avs = []
response_times = []
missed_trials = []
performance = []

mice = value_change_mice['tail'] + value_change_mice['Nacc']
print(mice)
for mouse_num, mouse_id in enumerate(mice):
    all_experiments = get_all_experimental_records()
    sessions = all_experiments[(all_experiments['mouse_id'] == mouse_id) & (all_experiments['experiment_notes'] == 'value switch')]['date'].values
    mouse_moving_avs = []
    mouse_response_times = []
    mouse_missed_trials = []
    mouse_performance = []
    for date in sessions:
        experiment_to_process = all_experiments[(all_experiments['date'] == date) & (all_experiments['mouse_id'] == mouse_id)]
        trial_data = open_experiment_just_behaviour(experiment_to_process, root_dir=repro_dir)
        red_trial_data = trial_data[trial_data['State name'] == 'TrialStart']
        red_trial_data_for_missed_trials = trial_data[trial_data['State name'] == 'TrialStart']
        red_trial_data_for_missed_trials.loc[trial_data['Trial outcome'] == 1, 'Trial outcome'] = 0
        red_trial_data_for_missed_trials.loc[trial_data['Trial outcome'] == 3, 'Trial outcome'] = 1
        red_trial_data.loc[trial_data['Trial outcome'] == 3, 'Trial outcome'] = 0
        response_trial_data = trial_data[trial_data['State name'] == 'WaitForResponse']
        post_trials = red_trial_data[red_trial_data['Trial num'] >= 100]
        pre_trials = red_trial_data[red_trial_data['Trial num'] < 100]
        post_reward_side = post_trials['Reward block'].unique()[0]
        if post_reward_side == 1: #left
            side = -1
        elif post_reward_side == 5: #right
            side = 1
        mouse_moving_avs.append(CalculateRBiasWindow(red_trial_data['First response'].reset_index(drop=True), red_trial_data['First choice correct'].reset_index(drop=True), 20) *side*100)
        mouse_response_times.append(moving_average(response_trial_data['Time end'].values - response_trial_data['Time start'].values, window=20))
        mouse_missed_trials.append(moving_average(red_trial_data_for_missed_trials['Trial outcome'].values, window=20) * 100, )
        mouse_performance.append(moving_average(red_trial_data['Trial outcome'].values, window=20) * 100)
    response_times.append(calc_per_mouse_mean(mouse_response_times))
    moving_avs.append(calc_per_mouse_mean(mouse_moving_avs))
    performance.append(calc_per_mouse_mean(mouse_performance))
    missed_trials.append(calc_per_mouse_mean(mouse_missed_trials))
    

['SNL_photo70', 'SNL_photo72', 'SNL_photo37', 'SNL_photo43', 'SNL_photo44', 'SNL_photo30', 'SNL_photo31', 'SNL_photo32', 'SNL_photo34', 'SNL_photo35']


C:\Users\francescag\AppData\Local\Temp\ipykernel_1308\421496107.py:9: RuntimeWarning: Mean of empty slice
  moving_av = np.nanmean(all_sessions, axis=0)
C:\Users\francescag\AppData\Local\Temp\ipykernel_1308\421496107.py:9: RuntimeWarning: Mean of empty slice
  moving_av = np.nanmean(all_sessions, axis=0)
C:\Users\francescag\AppData\Local\Temp\ipykernel_1308\421496107.py:9: RuntimeWarning: Mean of empty slice
  moving_av = np.nanmean(all_sessions, axis=0)
C:\Users\francescag\AppData\Local\Temp\ipykernel_1308\421496107.py:9: RuntimeWarning: Mean of empty slice
  moving_av = np.nanmean(all_sessions, axis=0)
C:\Users\francescag\AppData\Local\Temp\ipykernel_1308\421496107.py:9: RuntimeWarning: Mean of empty slice
  moving_av = np.nanmean(all_sessions, axis=0)
C:\Users\francescag\AppData\Local\Temp\ipykernel_1308\421496107.py:9: RuntimeWarning: Mean of empty slice
  moving_av = np.nanmean(all_sessions, axis=0)
C:\Users\francescag\AppData\Local\Temp\ipykernel_1308\421496107.py:9: RuntimeWarni

In [22]:
post_trials['Trial outcome'].mean()

0.9366197183098591

In [23]:
pre_trials['Trial outcome'].mean()

0.98

In [25]:
def plot_rolling_mean(var, mice, moving_avs, y_axis_label='% bias to big reward side'):
    
    all_mice, min_num_trials = align_mulitple_mice_moving_avs(mice, moving_avs)
    error_bar_lower, error_bar_upper = calculate_error_bars(np.nanmean(all_mice, axis=0),
                                                                     all_mice,
                                                                    error_bar_method='sem')
    font = {'size': 8}
    matplotlib.rc('font', **font)
    matplotlib.rcParams['pdf.fonttype'] = 42
    matplotlib.rcParams['font.sans-serif'] = 'Arial'
    matplotlib.rcParams['font.family']


    fig, axs = plt.subplots(1, 1, figsize=[2.5, 2])
    [axs.plot(m[:min_num_trials], alpha=0.5, c='gray', lw=0.5, label=f'mouse {i}') for i, m in enumerate(moving_avs)]
    axs.plot(np.nanmean(all_mice,axis=0), c='#5e8c89', lw=1, label='mean')
    axs.fill_between(np.arange(0, min_num_trials), error_bar_lower, error_bar_upper, alpha=0.5,
                                facecolor='#7FB5B5', linewidth=0)

    axs.axvline(100, color='k', label='block switch')
    axs.set_xlabel('trial number')
    axs.set_ylabel(y_axis_label)
    axs.spines['right'].set_visible(False)
    axs.spines['top'].set_visible(False)
    plt.tight_layout()
    plt.tight_layout()
    plt.savefig(os.path.join(figure_directory, '{} developing over trials.pdf'.format(var)))

    plt.show()
    return fig

# Rolling mean bias

In [29]:
bias_fig = plot_rolling_mean('bias', mice, moving_avs, y_axis_label='% bias to big reward side')
bias_xl = os.path.join(spreadsheet_path, 'ED_fig6', 'ED_fig6Y_bias_value_change.xlsx')
if not os.path.exists(bias_xl):
    save_figure_data_to_excel(bias_fig, bias_xl)

C:\Users\francescag\AppData\Local\Temp\ipykernel_1308\3170320309.py:4: RuntimeWarning: Mean of empty slice
  error_bar_lower, error_bar_upper = calculate_error_bars(np.nanmean(all_mice, axis=0),
C:\Users\francescag\AppData\Local\Temp\ipykernel_1308\3170320309.py:16: RuntimeWarning: Mean of empty slice
  axs.plot(np.nanmean(all_mice,axis=0), c='#5e8c89', lw=1, label='mean')


Data has been saved to S:\projects\APE_data_francesca_for_paper\spreadsheets_for_nature\ED_fig6\ED_fig6Y_bias_value_change.xlsx


# Rolling mean response time

In [30]:
response_time_fig = plot_rolling_mean('response time', mice, response_times, y_axis_label='response time (s)')
response_time_xl = os.path.join(spreadsheet_path, 'ED_fig6', 'ED_fig6X_response_time_state_change.xlsx')
if not os.path.exists(response_time_xl):
    save_figure_data_to_excel(response_time_fig, response_time_xl)

C:\Users\francescag\AppData\Local\Temp\ipykernel_1308\3170320309.py:4: RuntimeWarning: Mean of empty slice
  error_bar_lower, error_bar_upper = calculate_error_bars(np.nanmean(all_mice, axis=0),
C:\Users\francescag\AppData\Local\Temp\ipykernel_1308\3170320309.py:16: RuntimeWarning: Mean of empty slice
  axs.plot(np.nanmean(all_mice,axis=0), c='#5e8c89', lw=1, label='mean')


Data has been saved to S:\projects\APE_data_francesca_for_paper\spreadsheets_for_nature\ED_fig6\ED_fig6X_response_time_state_change.xlsx


In [31]:
performance_fig = plot_rolling_mean('performance', mice, performance, y_axis_label='% correct')
performance_xl = os.path.join(spreadsheet_path, 'ED_fig6', 'ED_fig6W_performance_state_change.xlsx')
if not os.path.exists(performance_xl):
    save_figure_data_to_excel(performance_fig, performance_xl)

C:\Users\francescag\AppData\Local\Temp\ipykernel_1308\3170320309.py:4: RuntimeWarning: Mean of empty slice
  error_bar_lower, error_bar_upper = calculate_error_bars(np.nanmean(all_mice, axis=0),
C:\Users\francescag\AppData\Local\Temp\ipykernel_1308\3170320309.py:16: RuntimeWarning: Mean of empty slice
  axs.plot(np.nanmean(all_mice,axis=0), c='#5e8c89', lw=1, label='mean')


Data has been saved to S:\projects\APE_data_francesca_for_paper\spreadsheets_for_nature\ED_fig6\ED_fig6W_performance_state_change.xlsx


In [32]:
missed_fig = plot_rolling_mean('missed trials', mice, missed_trials, y_axis_label='% missed trials')
missed_xl = os.path.join(spreadsheet_path, 'ED_fig6', 'ED_fig6V_missed_state_change.xlsx')
if not os.path.exists(missed_xl):
    save_figure_data_to_excel(missed_fig, missed_xl)

C:\Users\francescag\AppData\Local\Temp\ipykernel_1308\3170320309.py:4: RuntimeWarning: Mean of empty slice
  error_bar_lower, error_bar_upper = calculate_error_bars(np.nanmean(all_mice, axis=0),
C:\Users\francescag\AppData\Local\Temp\ipykernel_1308\3170320309.py:16: RuntimeWarning: Mean of empty slice
  axs.plot(np.nanmean(all_mice,axis=0), c='#5e8c89', lw=1, label='mean')


Data has been saved to S:\projects\APE_data_francesca_for_paper\spreadsheets_for_nature\ED_fig6\ED_fig6V_missed_state_change.xlsx
